In [ ]:
import pyspark
import glob
import pandas as pd



if 'sc' not in locals():
    from pyspark.context import SparkContext
    from pyspark.sql.context import SQLContext
    from pyspark.sql.session import SparkSession
    
    sc = SparkContext()
    sqlContext = SQLContext(sc)
    spark = SparkSession(sc)

spark

In [ ]:
print(glob.glob("data/NBA*"))

In [ ]:
mypath = './data'
files =  glob.glob("data/NBA*")


for file in files:
    df = pd.read_csv(file)
    if "Unnamed: 40" in df.columns:
        df = df.drop(["Unnamed: 40"], axis=1)

In [ ]:
from pyspark.sql.types import *

# Auxiliar functions
def equivalent_type(f):
    if f == 'datetime64[ns]': return TimestampType()
    elif f == 'int64': return LongType()
    elif f == 'int32': return IntegerType()
    elif f == 'float64': return FloatType()
    else: return StringType()

def define_structure(string, format_type):
    try: typo = equivalent_type(format_type)
    except: typo = StringType()
    return StructField(string, typo)

# Given pandas dataframe, it will return a spark's dataframe.
def pandas_to_spark(pandas_df):
    columns = list(pandas_df.columns)
    types = list(pandas_df.dtypes)
    struct_list = []
    for column, typo in zip(columns, types): 
      struct_list.append(define_structure(column, typo))
    p_schema = StructType(struct_list)
    return sqlContext.createDataFrame(pandas_df, p_schema)

In [ ]:
mypath = './data'
files =  glob.glob("data/NBA*")

spark_table_games = "games"
for file in files:
    df = pd.read_csv(file)
    if "Unnamed: 40" in df.columns:
        df = df.drop(["Unnamed: 40"], axis=1)
    sparkDF=spark.createDataFrame(df) 
    sparkDF.write.format("parquet").saveAsTable(spark_table_games, mode='append')

In [ ]:
type(df)

In [ ]:
spark.sql(f"drop table games")

In [ ]:
type(spark_df)

In [ ]:
csv_files = glob.glob("data/NBA*")

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_games = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(csv_files))


In [ ]:
df_games.printSchema()

In [ ]:
# create table
spark_table_games = "games"
df_games.write.format("parquet").saveAsTable(spark_table_games)

In [ ]:
spark.sql("SELECT count(*) from games").collect()

In [ ]:
###################################
# Using database below

In [1]:
from databaseClass import DB
import utils
import sql_files
import numpy as np
import pandas as pd

In [2]:
userName = utils.userName
userPass = utils.userPass
dbName = utils.dbName


In [3]:
db = DB(userName = userName, userPass = userPass, dataBaseName = dbName)

In [4]:
def generate_sql(input_sql_file):
    with open(input_sql_file, 'r') as file:
        sql = file.read()
        return sql
    
def build_table(sql_file, table_name):
    '''postgres does not have a create or replace table option.  so we do that here instead'''
    try:
        db.BuildTableFromQuery(generate_sql(sql_file), table_name)
    except:
        db.dropTable(table_name)
        db.BuildTableFromQuery(generate_sql(sql_file), table_name)

In [ ]:
build_table('sql_files/game_scores.sql', 'game_scores')

In [ ]:
build_table('sql_files/shot_performance.sql', 'player_game_shot_performance')

In [ ]:
build_table('sql_files/game_rosters_1.sql', 'game_rosters_1')

In [ ]:
build_table('sql_files/game_rosters_2.sql', 'game_rosters_2')

In [ ]:
build_table('sql_files/game_rebounds.sql', 'game_rebounds')

In [ ]:
build_table('sql_files/output.sql', 'output')

In [ ]:
db.DBtoDF(generate_sql('sql_files/shot_performance.sql'))

In [ ]:
df = db.DBtoDF(generate_sql('sql_files/test.sql'))

In [ ]:
df['regexp_match'].iloc[1][0]

In [ ]:
##################

df_games = db.DBtoDF(generate_sql('sql_files/output.sql'))

In [ ]:
df_games.info()

In [ ]:
df_games[df_games['team'].notnull()]

In [ ]:
df_games['player'] = df_games['player'].str[0]

In [ ]:
df_games['player_team'] = df_games['player'] #this cell only used for testing

In [ ]:
df_games['player_team'] = df_games['player'] + "_" + df_games['team']

In [ ]:
df_games.head(20)

In [ ]:
df_games[df_games['player_team'] == 'A. Baynes_DET'].sort_values('url')

In [ ]:
df_temp = df_games[df_games['player_team'] == 'A. Baynes_DET'].drop(columns=['homescore', 'awayscore'])

In [ ]:
def get_data():
    df_games = db.DBtoDF(generate_sql('sql_files/output.sql'))
    df_games = df_games[df_games['team'].notnull()]
    df_games['player'] = df_games['player'].str[0]
    df_games['player_team'] = df_games['player'] + "_" + df_games['team']
    player_names = df_games['player_team'].unique()
    player_names = player_names[~pd.isnull(player_names)] #remove empty player names
    return df_games, player_names

def pre_process_df(player_team):
    df_temp = df_games[df_games['player_team'] == player_team].drop(columns=['homescore', 'awayscore'])
    df_temp = df_temp.rolling(window=3)
    df_temp = df_temp.mean().round(decimals=2)
    df_temp['player_team'] = player_team
    return df_temp

df_games, player_names = get_data()

df_list = []
for player in player_names:
    df_list.append(pre_process_df(player))

df = pd.concat(df_list, axis=0)
df_merged = df_games.merge(df, how='outer', left_index=True, right_index=True)
df_merged = df_merged[~df_merged['two_point_shots_made_y'].isnull()]
df_merged['spread_target'] = df_merged['homescore'] - df_merged['awayscore']

columns_to_drop = ['date', 'player','two_point_shots_made_x', 
                      'two_point_shots_x', 'three_point_shots_made_x',
                      'three_point_shots_made_x', 'three_point_shots_x',
                      'total_makes_x', 'total_shots_x', 'defensive_rebounds_x',
                      'offensive_rebounds_x', 'homescore', 'awayscore', 'player_team_x'
                     ]

df_merged = df_merged.drop(columns_to_drop, axis=1)
# at this stage, we've sent the final data to postgres.  next step is back to python, then pivot and prep for modeling


In [13]:
df_final = db.DBtoDF(generate_sql('sql_files/final.sql'))

#values are the features we plan to use for modeling
values = ['two_point_shots_made_y','two_point_shots_y',
          'three_point_shots_made_y','three_point_shots_y',
          'total_makes_y','total_shots_y','defensive_rebounds_y',
          'offensive_rebounds_y', 'spread_target']

#df is now going to be in the right shape for modeling
df_final = pd.pivot_table(df_final, values=values, 
                                   columns=['player_team_y'],
                                   index='url',
                                  aggfunc=np.mean)
#collapsing the pivot multi index
df_final.columns = df_final.columns.to_series().str.join('_')

In [14]:
df_final.head()

,defensive_rebounds_y_A. Afflalo_NYK,defensive_rebounds_y_A. Afflalo_ORL,defensive_rebounds_y_A. Afflalo_SAC,defensive_rebounds_y_A. Aminu_ORL,defensive_rebounds_y_A. Aminu_POR,defensive_rebounds_y_A. Anderson_LAC,defensive_rebounds_y_A. Anderson_WAS,defensive_rebounds_y_A. Bargnani_BRK,defensive_rebounds_y_A. Baynes_BOS,defensive_rebounds_y_A. Baynes_DET,...,two_point_shots_y_Z. La_MIN,two_point_shots_y_Z. Norvell_GSW,two_point_shots_y_Z. Pachulia_DAL,two_point_shots_y_Z. Pachulia_DET,two_point_shots_y_Z. Pachulia_GSW,two_point_shots_y_Z. Qi_HOU,two_point_shots_y_Z. Randolph_MEM,two_point_shots_y_Z. Randolph_SAC,two_point_shots_y_Z. Smith_PHI,two_point_shots_y_Z. Williamson_NOP
url,,,,,,,,,,,,,,,,,,,,,
/boxscores/201510300ATL.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/boxscores/201510300CLE.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/boxscores/201510300DET.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/boxscores/201510310IND.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/boxscores/201510310LAC.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
pd.pivot_table(df_merged, index='url', 
               
               
               
               columns=['player_team'], aggfunc=np.mean)

In [ ]:
df_merged.head()

In [ ]:
df_merged.info()

In [ ]:
df_merged = df_merged[~df_merged['two_point_shots_made_y'].isnull()]

In [ ]:
df_merged.to_csv('output.csv')

In [ ]:
df.sort_index().to_csv('output.csv')

In [ ]:
df_list[0].shape

In [ ]:
player_names[0:5]

In [ ]:
player_names = player_names[~pd.isnull(player_names)] #remove empty player names

In [ ]:
df

In [ ]:
for element in df_list:
    print(element.shape)

In [ ]:
df_games_grouped = pd.pivot_table(df_games, values=['two_point_shots_made', 'two_point_shots'], 
                                   columns=['player_team'],
                                   index='url',
                                  aggfunc=np.mean
                                  ).rolling(3).mean()